In [1]:
import tensorflow as tf
import keras
import os 
import cv2

import numpy as np
from sklearn.metrics import accuracy_score
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from sklearn.model_selection import train_test_split
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip,RandomRotation

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import DepthwiseConv2D, Conv2D, Dense,BatchNormalization,Activation,Input,AveragePooling2D,Flatten,Dropout

In [2]:
def SepaableConv(x,num_filters, strides, alpha=1.0):
    x=DepthwiseConv2D(kernel_size=3,padding='same')(x)
    x=BatchNormalization(momentum=0.9998)(x)
    x=Activation('relu')(x)
    x=Conv2D(np.floor(num_filters * alpha),kernel_size=(1,1),strides=strides,use_bias=False,padding='same')(x)
    x=BatchNormalization(momentum=0.9998)(x)
    x=Activation('relu')(x)
    return x

In [3]:
def Conv(x,num_filters,kernel_size,strides=1,alpha=1.0):
    x = Conv2D(np.floor(num_filters * alpha),kernel_size=(1,1),strides=strides,use_bias=False,padding='same')(x)
    x = BatchNormalization(momentum=0.9998)(x)
    x = Activation('relu')(x)
    
    return x 

In [4]:
num_classes=2
inputs = Input(shape=(224,224,3))

In [5]:
x= Conv(inputs,num_filters=32,kernel_size=3,strides=2)
x= SepaableConv(x,num_filters=32,strides=1)

x= Conv(x ,num_filters=64,kernel_size=1,)
x= SepaableConv(x,num_filters=64,strides=2)

x= Conv(x ,num_filters=128,kernel_size=1)
x= SepaableConv(x,num_filters=128,strides=1)

x= Conv(x ,num_filters=256,kernel_size=1)
x= SepaableConv(x,num_filters=256,strides=1)


x= Conv(x ,num_filters=256,kernel_size=1)
x= SepaableConv(x,num_filters=256,strides=2)

x= Conv(x ,num_filters=512,kernel_size=1)

for i in range(5):  
    x= SepaableConv(x,num_filters=512,strides=1)
    x= Conv(x ,num_filters=512,kernel_size=1)
    
x= SepaableConv(x,num_filters=512,strides=1)

x= Conv(x ,num_filters=1024,kernel_size=1)

x=AveragePooling2D(pool_size=(7,7))(x)
x=Flatten()(x)

outputs = Dense(num_classes,activation='softmax')(x)
# outputs = Dropout(0.1)(outputs)
outputs = Activation('softmax')(outputs)

In [6]:
model=Model(inputs, outputs)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=Adam(0.001),
             metrics=['acc'])
# model.compile(optimizer = Adam(lr = 0.0001),
#                   loss='mse',
#                   metrics = ["auc"])

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      96        
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 32)     128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 112, 112, 32)     320       
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 32)     128   

In [8]:
datasets_directory='corona/'
categories=['Negative','Positive']
img_size=224

In [9]:
image_augmentation=Sequential([RandomFlip('horizontal_and_vertical'),
                             RandomRotation(0.2)])

In [10]:
trainingDatasets=[]
def create_trainingDatasets():
    for i in categories:
        path = os.path.join(datasets_directory,i)
        class_number=categories.index(i)
        
        for img in os.listdir(path):
            imageArray= cv2.imread(os.path.join(path,img))
            img_size=224
            modifiedArray=cv2.resize(imageArray,(img_size,img_size))
            modifiedArray=tf.expand_dims(modifiedArray,0)
            
        for i in range(400):
            imgAugment = image_augmentation(modifiedArray)
            trainingDatasets.append([modifiedArray,class_number])
            
create_trainingDatasets()

In [11]:
print(len(trainingDatasets)) 

800


In [12]:
XDatasets = []
yDatasets = []

for feature, labels in trainingDatasets:
    XDatasets.append(feature)
    yDatasets.append(labels)

In [13]:
XDatasets =np.array(XDatasets).reshape(-1,img_size,img_size,3)
yDatasets = np.array(yDatasets).reshape((-1,1))


In [14]:
XDatasets = XDatasets.astype('float32')
XDatasets/=255

In [15]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(XDatasets,yDatasets,test_size=0.2,random_state=4)

In [16]:
model.fit(xtrain,ytrain,epochs=3,validation_data=(xtest,ytest),shuffle=True)

Epoch 1/3


ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv2d_22/Conv2D' defined at (most recent call last):
    File "C:\Users\Md. Mahabur Rahman\anaconda3\envs\MobileNet\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "C:\Users\Md. Mahabur Rahman\anaconda3\envs\MobileNet\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "C:\Users\Md. Mahabur Rahman\anaconda3\envs\MobileNet\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Md. Mahabur Rahman\anaconda3\envs\MobileNet\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Md. Mahabur Rahman\anaconda3\envs\MobileNet\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Md. Mahabur Rahman\anaconda3\envs\MobileNet\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
   